In [6]:
import os
import cv2
import glob
import numpy as np
import pickle
import torch
from tqdm import tqdm
from ultralytics import YOLO
from sklearn.linear_model import LinearRegression
import time
# --- Config ---
IMAGE_DIR = "/workspace/hjs/python/UA-DETRAC_UPD_ANN/images/train"
MODEL_PATH = "yolov8n.pt"
TRACKER_CONFIG = "bytetrack.yaml"
HOMOGRAPHY_PATH = "/workspace/hjs/python/temp/homography.npy"
OUTPUT_PATH = "car_trajectories2.pkl"
TARGET_CLASS = "car"
CONF_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5

# --- Check for GPU ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# --- Load homography ---
H = np.load(HOMOGRAPHY_PATH).astype(np.float32)
def img_to_world(pt):
    px = np.array([pt[0], pt[1], 1.0], dtype=np.float32)
    wx = np.dot(H, px)
    wx /= wx[2]
    return wx[0], wx[1]

# --- Simple Kalman filter (constant velocity) ---
class KalmanFilter:
    def __init__(self, x, y):
        self.state = np.array([x, y, 0, 0], dtype='float32')
        self.P = np.eye(4) * 500
        dt = 1
        self.F = np.array([[1, 0, dt, 0],
                           [0, 1, 0, dt],
                           [0, 0, 1,  0],
                           [0, 0, 0,  1]], dtype='float32')
        self.H = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0]], dtype='float32')
        self.Q = np.eye(4) * 0.01
        self.R = np.eye(2) * 0.1

    def update(self, x, y):
        self.state = self.F @ self.state
        self.P = self.F @ self.P @ self.F.T + self.Q
        z = np.array([x, y], dtype=np.float32)
        y_ = z - self.H @ self.state
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.state += K @ y_
        self.P = (np.eye(4) - K @ self.H) @ self.P
        return self.state[0], self.state[1]

# --- Load YOLO model (Ultralytics uses GPU automatically) ---
model = YOLO(MODEL_PATH)

# --- Get class ID for 'car' ---
class_names = model.model.names
target_class_id = [i for i, name in class_names.items() if name.lower() == TARGET_CLASS.lower()]
if not target_class_id:
    raise ValueError(f"Class '{TARGET_CLASS}' not found in model classes.")
target_class_id = target_class_id[0]

# --- Load image paths ---
image_paths = sorted(glob.glob(os.path.join(IMAGE_DIR, "*.jpg")))
fps = 25  # UA-DETRAC default
kalman_filters = {}
vehicle_trajectories = {}
track_last_seen = {}

# --- Process each frame with progress bar ---
for frame_idx, path in enumerate(tqdm(image_paths, desc="Processing frames")):
    start_time = time.time()

    frame = cv2.imread(path)
    results = model.track(
        frame,
        tracker=TRACKER_CONFIG,
        persist=True,
        conf=CONF_THRESHOLD,
        iou=IOU_THRESHOLD,
        verbose=False,
        device=device
    )[0]

    if results.boxes is None:
        continue

    for box in results.boxes:
        cls_id = int(box.cls[0].item())
        conf = box.conf[0].item()
        if cls_id != target_class_id or conf < CONF_THRESHOLD:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        track_id = int(box.id[0].item()) if box.id is not None else -1
        cx, cy = (x1 + x2) // 2, int(y2)

        if track_id not in kalman_filters:
            kalman_filters[track_id] = KalmanFilter(cx, cy)
        smoothed_cx, smoothed_cy = kalman_filters[track_id].update(cx, cy)

        wx, wy = img_to_world((smoothed_cx, smoothed_cy))
        vehicle_trajectories.setdefault(track_id, []).append((frame_idx, wx, wy))
        track_last_seen[track_id] = frame_idx

    # Remove stale tracks
    for tid in list(track_last_seen):
        if frame_idx - track_last_seen[tid] > 30:
            track_last_seen.pop(tid)
            kalman_filters.pop(tid)

    # Optional: Print time per frame
    elapsed = time.time() - start_time
#     print(f"Processed frame {frame_idx + 1}/{len(image_paths)} in {elapsed:.2f} sec", end='\r')

# --- Final report ---
print(f"\nCollected {len(vehicle_trajectories)} tracks")

# --- Filter short tracks ---
min_length = 5
vehicle_trajectories = {
    k: v for k, v in vehicle_trajectories.items() if len(v) >= min_length
}

# --- Save to file ---
with open(OUTPUT_PATH, "wb") as f:
    pickle.dump(vehicle_trajectories, f)

print(f"Saved {len(vehicle_trajectories)} valid trajectories to {OUTPUT_PATH}")


Using device: cuda


Processing frames:   9%|▉         | 7802/83791 [01:25<13:57, 90.77it/s]


KeyboardInterrupt: 

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

# --- Load data ---
with open("car_trajectories.pkl", "rb") as f:
    data = pickle.load(f)
print('unloaded')
# --- Parameters ---
input_len = 10   # use past 10 positions
pred_len = 5     # predict next 5 positions

X_seqs = []
Y_seqs = []

for track_id, points in data.items():
    coords = np.array([[p[1], p[2]] for p in points])  # drop frame index
    if len(coords) < input_len + pred_len:
        continue
    for i in range(len(coords) - input_len - pred_len + 1):
        input_seq = coords[i:i+input_len]
        pred_seq = coords[i+input_len:i+input_len+pred_len]
        X_seqs.append(input_seq)
        Y_seqs.append(pred_seq)

X = np.array(X_seqs)  # (N, 10, 2)
Y = np.array(Y_seqs)  # (N, 5, 2)

print(f"Prepared dataset: X={X.shape}, Y={Y.shape}")
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


unloaded
Prepared dataset: X=(609284, 10, 2), Y=(609284, 5, 2)


In [4]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

# --- Load trajectory data ---
with open("car_trajectories.pkl", "rb") as f:
    data = pickle.load(f)

# --- Parameters ---
input_len = 10
pred_len = 5
X_seqs, Y_seqs = [], []

for track_id, points in data.items():
    coords = np.array([[p[1], p[2]] for p in points])  # remove frame index
    if len(coords) < input_len + pred_len:
        continue
    for i in range(len(coords) - input_len - pred_len + 1):
        X_seqs.append(coords[i:i+input_len])
        Y_seqs.append(coords[i+input_len:i+input_len+pred_len])

X = np.array(X_seqs)  # shape: [N, 10, 2]
Y = np.array(Y_seqs)  # shape: [N, 5, 2]
print(f"Prepared dataset: X={X.shape}, Y={Y.shape}")

# --- Train/Val Split ---
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# --- GPU ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- DataLoaders ---
BATCH_SIZE = 64
train_ds = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
val_ds = TensorDataset(torch.Tensor(X_val), torch.Tensor(Y_val))
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

# --- GRU Model ---
class GRU(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=1, output_len=5, dropout=0.1):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_len * 2)

    def forward(self, x):  # x: [B, 10, 2]
        _, hn = self.gru(x)              # hn: [num_layers, B, hidden]
        out = self.fc(hn[-1])            # out: [B, output_len * 2]
        return out.view(-1, 5, 2)        # reshape to [B, 5, 2]

model = GRU().to(device)

# --- Optimizer, Loss ---
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
mse_loss = nn.MSELoss()

# --- Custom velocity loss ---
def velocity_loss(pred, target):
    pred_vel = pred[:, 1:] - pred[:, :-1]
    target_vel = target[:, 1:] - target[:, :-1]
    return nn.functional.mse_loss(pred_vel, target_vel)

# --- Evaluation Metrics ---
def compute_metrics(pred, target):
    ade = torch.mean(torch.norm(pred - target, dim=2))  # average displacement error
    fde = torch.mean(torch.norm(pred[:, -1] - target[:, -1], dim=1))  # final displacement error
    mae = torch.mean(torch.abs(pred - target))
    return ade.item(), fde.item(), mae.item()

# --- Training Loop ---
best_val_loss = float('inf')
patience = 5
epochs_without_improvement = 0
checkpoint_path = 'best_gru_model.pt'
EPOCHS = 100


Prepared dataset: X=(609284, 10, 2), Y=(609284, 5, 2)
Using device: cuda


In [5]:
best_val_loss = float('inf')
patience = 5
epochs_without_improvement = 0
checkpoint_path = 'best_gru_model.pt'
EPOCHS = 100  # Set higher, since early stopping will control actual training duration

for epoch in range(EPOCHS):
    print(f"\n--- Starting Epoch {epoch + 1}/{EPOCHS} ---")

    model.train()
    train_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item() * xb.size(0)

    model.eval()
    val_loss, ade, fde, mae = 0, 0, 0, 0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
            val_loss += loss.item() * xb.size(0)
            batch_ade, batch_fde, batch_mae = compute_metrics(pred, yb)
            ade += batch_ade * xb.size(0)
            fde += batch_fde * xb.size(0)
            mae += batch_mae * xb.size(0)

    train_loss /= len(train_dl.dataset)
    val_loss /= len(val_dl.dataset)
    ade /= len(val_dl.dataset)
    fde /= len(val_dl.dataset)
    mae /= len(val_dl.dataset)
    scheduler.step(val_loss)

    print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} | "
          f"ADE: {ade:.4f}, FDE: {fde:.4f}, MAE: {mae:.4f}")

    # --- Early Stopping Check ---
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print("✔️  New best model saved.")
    else:
        epochs_without_improvement += 1
        print(f"⚠️  No improvement. {epochs_without_improvement} epochs without improvement.")

    if epochs_without_improvement >= patience:
        print("⏹️ Early stopping triggered.")
        break



--- Starting Epoch 1/100 ---
Epoch 1 - Train Loss: 0.3056, Val Loss: 0.0173 | ADE: 0.1212, FDE: 0.1544, MAE: 0.0694
✔️  New best model saved.

--- Starting Epoch 2/100 ---
Epoch 2 - Train Loss: 0.0115, Val Loss: 0.0115 | ADE: 0.0678, FDE: 0.1027, MAE: 0.0420
✔️  New best model saved.

--- Starting Epoch 3/100 ---
Epoch 3 - Train Loss: 0.0107, Val Loss: 0.0124 | ADE: 0.0860, FDE: 0.1166, MAE: 0.0498
⚠️  No improvement. 1 epochs without improvement.

--- Starting Epoch 4/100 ---
Epoch 4 - Train Loss: 0.0098, Val Loss: 0.0099 | ADE: 0.0665, FDE: 0.0890, MAE: 0.0390
✔️  New best model saved.

--- Starting Epoch 5/100 ---
Epoch 5 - Train Loss: 0.0092, Val Loss: 0.0087 | ADE: 0.0470, FDE: 0.0735, MAE: 0.0276
✔️  New best model saved.

--- Starting Epoch 6/100 ---
Epoch 6 - Train Loss: 0.0088, Val Loss: 0.0098 | ADE: 0.0650, FDE: 0.1044, MAE: 0.0368
⚠️  No improvement. 1 epochs without improvement.

--- Starting Epoch 7/100 ---
Epoch 7 - Train Loss: 0.0086, Val Loss: 0.0092 | ADE: 0.0506, FD

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
